In [76]:
import pandas as pd
import requests
import json
import math as m

key = '2988f01f5e86175bda8beae2b5035e1ccef2d052'

url = f"https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&key={key}&for=state:*"

JSONContent = requests.get(url).json()

states = pd.DataFrame(JSONContent)
states.columns = states.iloc[0]
states = states.iloc[1:,2]
states = [int(i) for i in states]
states = sorted(states)

In [77]:
columns=['GEO_ID','FSRR2010']

responses = pd.DataFrame(columns=columns)

for i in states:
    if i < 10:
        url = f"https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&key={key}&for=tract:*&in=state:0"\
        + str(i)
    else:
        url = f"https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&key={key}&for=tract:*&in=state:"\
        + str(i)
            
    JSONContent = requests.get(url).json()        
    temp = pd.DataFrame(JSONContent)
    temp = temp.iloc[1:,[0,1]]
    temp.columns = columns
    responses = pd.concat([responses,temp])

In [78]:
url_base = f'https://api.census.gov/data/2010/dec/sf1?get=\
GEO_ID,P001001,P002005,P003002,P003003,P003004,\
P003005,P003006,P003007,P003008,P004003,\
P016003,P037001&key={key}&for=tract:*&in=state:'
url = url_base + '01'
JSONContent = requests.get(url).json()
temp = pd.DataFrame(JSONContent)
columns = temp.iloc[0,:-3]
dec_demos = pd.DataFrame(columns = columns)

for i in states:
    if i<10:
        url = url_base + '0' + str(i)
    else:
        url = url_base + str(i)

    JSONContent = requests.get(url).json()
    temp = pd.DataFrame(JSONContent)
    temp = temp.iloc[1:,:-3]
    temp.columns=columns
    dec_demos = pd.concat([dec_demos,temp])

In [79]:
url_base = f'https://api.census.gov/data/2010/acs/acs5?get=GEO_ID,\
B05001_006E,B06007_033E,B06007_005E,B06007_008E,\
B06009_002E,B17010_002E,B06010_011E&key={key}&for=tract:*&in=state:'
url = url_base + '01'
JSONContent = requests.get(url).json()
temp = pd.DataFrame(JSONContent)
columns = temp.iloc[0,:-3]
acs_demos = pd.DataFrame(columns = columns)

for i in states:
    if i<10:
        url = url_base + '0' + str(i)
    else:
        url = url_base + str(i)
    
    JSONContent = requests.get(url).json()   
    temp = pd.DataFrame(JSONContent)
    temp = temp.iloc[1:,:-3]
    temp.columns=columns
    acs_demos = pd.concat([acs_demos,temp])

In [80]:
dec_demos.index = dec_demos.GEO_ID
dec_demos = dec_demos.drop(columns='GEO_ID')
acs_demos.index = acs_demos.GEO_ID
acs_demos = acs_demos.drop(columns='GEO_ID')
responses.index = responses.GEO_ID
responses = responses.drop(columns='GEO_ID')

data = responses.merge(dec_demos,how='inner',left_index=True,right_index=True)
data = data.merge(acs_demos,how='inner',left_index=True,right_index=True)
data = data[data.FSRR2010.notna()]

data.columns = ['response','pop','rural','white',\
                'black','AIAN','asian','pacific_islander',\
                'other_race','multiracial','hispanic',\
               'under18','avgfamsize','noncitizen',\
               'foreign_born','spanish_poor_english','other_lang_poor_english',\
               'no_high_school','below_poverty_level','make_over_75k']

data = data.replace('None','0')

data = data.astype('float32')

for column in data.columns:
    if column not in ['pop','response','avgfamsize']:
        data[column] = data[column]/data['pop']

        
data = data.fillna(value='0')
data = data.replace(m.inf,0)

data.to_csv('2020responsesdataset.csv')

In [81]:
#search function for finding ACS variables

myfile = json.load(open('acs_variables.json'))

def search_dict(string):
    for dict in myfile['variables']:
        if string in str(myfile['variables'][dict]):
            print(dict,'\n',myfile['variables'][dict]['label'],'\n\n\n')
            
# example call to search: search_dict('poverty')

# example call to check data: myfile['variables']['B06010_011E']